# DB acute analysis
By Stephen Larroque @ Coma Science Group, GIGA Research, University of Liege
Creation date: 2018-05-27
License: MIT
v1.0.1

DESCRIPTION:
Calculate whether patients were acute at the time of MRI acquisition (28 days included by default).
This expects as input a csv file with both the accident date and dicom date (see other scripts).

INSTALL NOTE:
You need to pip install pandas before launching this script.
Tested on Python 2.7.13

USAGE:

TODO:


In [ ]:
# Forcefully autoreload all python modules
%load_ext autoreload
%autoreload 2

In [ ]:
# AUX FUNCTIONS

import os, sys

cur_path = os.path.realpath('.')
sys.path.append(os.path.join(cur_path, 'csg_fileutil_libs'))

import re

from csg_fileutil_libs.aux_funcs import save_df_as_csv, _tqdm, reorder_cols_df, find_columns_matching, convert_to_datetype


In [ ]:
# Nice plots!
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
# PARAMETERS

# FileMakerPro (FMP) database, cleaned with the provided script
fmp_agg_csv = 'fmp_db_subjects_aggregated.csv_etiosedatfixed_dicomsdatediag.csv'


In [ ]:
# Import the csv dbs as dataframes
import pandas as pd
import ast

cf_agg = pd.read_csv(fmp_agg_csv, sep=';', low_memory=False).dropna(axis=0, how='all')  # drop empty lines
cf_agg.set_index('Name', inplace=True)
cf_agg

In [ ]:
def df_extract_first_date(x):
    if not pd.isnull(x):
        try:
            x2 = ast.literal_eval(x)
        except SyntaxError as exc:
            x2 = ast.literal_eval("['"+x+"']")
        return x2[0].split(':')[0]
    else:
        return x
first_crsr_date = cf_agg['CRSr::Date and subscores'].apply(df_extract_first_date)
cf_agg['CRSr first date'] = first_crsr_date
cf_agg

In [ ]:
# Convert to datetime the columns we need, to ease date calculations
cf_agg2 = convert_to_datetype(cf_agg, 'Date of Accident', '%d/%m/%Y', errors='coerce')
cf_agg2 = convert_to_datetype(cf_agg2, 'CRSr first date', '%d/%m/%Y', errors='coerce')
cf_agg2 = convert_to_datetype(cf_agg2, 'Dicom Date', '%Y-%m-%d', errors='coerce')
cf_agg2

In [ ]:
# Acute from a random CRS-R date
cf_agg2['Days random CRSr since accident'] = cf_agg2['CRSr first date'] - cf_agg2['Date of Accident']
cf_agg2['Days random CRSr since accident']

In [ ]:
# Acute from dicom date
cf_agg2['Days scan since accident'] = cf_agg2['Dicom Date'] - cf_agg2['Date of Accident']
cf_agg2.loc[:, ['Name', 'CRSr::Best Computed Outcome', 'CRSr::Best Diagnosis', 'Final diagnosis', 'Days scan since accident']]
cf_agg2['Acute'] = (cf_agg2['Days scan since accident'] <= pd.Timedelta('28 days'))
# Nullify if no dicom date available (then cannot know if acute or not)
cf_agg2.loc[cf_agg2['Dicom Date'].isnull(), ['Days scan since accident', 'Days random CRSr since accident']] = None
cf_agg2.loc[cf_agg2['Dicom Date'].isnull(), 'Acute'] = ''
# Save as csv
save_df_as_csv(cf_agg2, fmp_agg_csv+'_acute.csv', fields_order=cf_agg2.columns, keep_index=False)
cf_agg2